# PyTorch Introduction

Most machine learning workflows involve working with data, creating models, optimizing model parameters, and saving the trained models. This tutorial introduces you to a complete ML workflow implemented in PyTorch, with links to learn more about each of these concepts

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Create a tensor from a list
tensor1 = torch.tensor([1, 2, 3])
print(tensor1)

# Create a tensor of zeros with shape (2, 3)
tensor2 = torch.zeros(2, 3)
print(tensor2)

# Create a random tensor with shape (3, 2)
tensor3 = torch.rand(3, 2)
print(tensor3)

# Performing operations on Tensors

# Addition
result_add = tensor1 + tensor2
print(result_add)


# Multiplication
result_mul = tensor2 * 5
print(result_mul)


# Matrix multiplication
result_matmul = torch.matmul(tensor2, tensor3)
print(result_matmul)

tensor([1, 2, 3])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[0.5942, 0.7193],
        [0.8057, 0.1604],
        [0.1688, 0.0891]])
tensor([[1., 2., 3.],
        [1., 2., 3.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
tensor([[0., 0.],
        [0., 0.]])


## Automatic Differentiation

In [4]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# Perform a computation
z = x**2 + y**3
print(z)

# Compute gradients
z.backward()
print(x.grad)
print(y.grad)

tensor(31., grad_fn=<AddBackward0>)
tensor(4.)
tensor(27.)


## Load Dataset

In [6]:
iris = load_iris()
X, y = iris.data, iris.target

## Split Dataset

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardize data

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Create model

In [10]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input layer
        self.relu = nn.ReLU()                          # Activation function
        self.fc2 = nn.Linear(hidden_size, output_size) # Output layer
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [11]:
# Set random seed for reproducibility
torch.manual_seed(42)


# Define the input size, hidden size, and output size of the neural network
input_size = X.shape[1]
hidden_size = 10
output_size = len(iris.target_names)


# Instantiate the neural network
model = SimpleNN(input_size, hidden_size, output_size)


# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Convert datto PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)

In [12]:
# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
   
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
   
    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.7783
Epoch [20/100], Loss: 0.5399
Epoch [30/100], Loss: 0.3921
Epoch [40/100], Loss: 0.2934
Epoch [50/100], Loss: 0.2166
Epoch [60/100], Loss: 0.1639
Epoch [70/100], Loss: 0.1284
Epoch [80/100], Loss: 0.1050
Epoch [90/100], Loss: 0.0902
Epoch [100/100], Loss: 0.0800


## Evaluate model

In [13]:
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test)
    y_test_tensor = torch.LongTensor(y_test)
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f'Accuracy on the test set: {accuracy:.2f}')

Accuracy on the test set: 0.97
